# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: Juan Bernardo Orozco Quirarte

**Professor**: Pablo Camarillo Ramirez

### Create Spark Session

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (files)") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/09 01:55:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Create Schema

In [2]:
from bernardoorozco.spark_utils import SparkUtils

logs_schema = SparkUtils.generate_schema([("user", "string"), ("timestamp", "string"),("endpoint", "string"),("status","int")])

raw_df = spark.readStream \
            .format("json") \
            .schema(logs_schema) \
            .load("/opt/spark/work-dir/data/logs/")

### Output

In [3]:
query = raw_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .start()

query.awaitTermination(30)  # Espera 30 segundos para mostrar al menos 3 micro-batches

-------------------------------------------
Batch: 0
-------------------------------------------
+--------+--------------------+--------+------+
|user    |timestamp           |endpoint|status|
+--------+--------------------+--------+------+
|Bernardo|08/10/2025, 19:55:44|/login  |404   |
+--------+--------------------+--------+------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+--------------------+--------+------+
|user |timestamp           |endpoint|status|
+-----+--------------------+--------+------+
|Diego|08/10/2025, 19:55:49|/admin  |200   |
+-----+--------------------+--------+------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+--------------------+--------+------+
|user    |timestamp           |endpoint|status|
+--------+--------------------+--------+------+
|Bernardo|08/10/2025, 19:55:54|/home   |502   |
+--------+--------------------+--------+------+

-

False

-------------------------------------------
Batch: 6
-------------------------------------------
+-----+--------------------+---------+------+
|user |timestamp           |endpoint |status|
+-----+--------------------+---------+------+
|Pablo|08/10/2025, 19:56:14|/api/data|502   |
+-----+--------------------+---------+------+

-------------------------------------------
Batch: 7
-------------------------------------------
+----+--------------------+--------+------+
|user|timestamp           |endpoint|status|
+----+--------------------+--------+------+
|Luis|08/10/2025, 19:56:19|/admin  |200   |
+----+--------------------+--------+------+



In [4]:
sc.stop()